In [48]:
import pandas as pd
import os 
import numpy as np
import pickle as pkl
import fileutils as fileutils



In [49]:
def alternate_labels_path(dataDrive,LabelNumber, symbol):
    """
    path of labels and symbol to store data
    labelNumber: label number pick from one two three
    symbol: symbol in capital letters and .L in the end
    """
    LabelAlternate = 'LabelsAlternate'+str(LabelNumber)
    
    path = os.path.join(dataDrive, LabelAlternate, str(symbol))
    if os.path.exists(path):
        return path
    else: 
        print('path does not exist- add a .L to the symbol ')

In [50]:
LabelsAlternateNames = ['LabelsAlternateOne',
 'LabelsAlternateTwo',
 'LabelsAlternateThree',
 'LabelsAlternateFour',
 'LabelsAlternateFive'] #names of alternate labels


### Paths ### 

In [67]:

# paths
dataDrive = '/media/ak/My Passport/Data/FinDataReal/'
extPath = '/media/ak/My Passport/ExperimentData'
featuresPath = "/".join((extPath, 'features')) # path with features
MKLPath = "/".join((extPath, 'MKLExpPath'))
labels_location_folder = fileutils.data_path # this is the folder where all the labels are saved

labels_pickle_files = [s for s in os.listdir(labels_location_folder ) if ('LabelsAlternate') in s] # these are all the dicts that we have alternate labels for. 

# ## symbols - getting all the symbols that I have alternate data for
# symbols  = [f.split("_")[0] for f in labels_pickle_files] 

In [73]:
symbols  = [f for f in [s for s in os.listdir(labels_location_folder ) if ('.L') in s] if ('.L_A') not in f]
symbols

['LLOY.L',
 'AAL.L',
 'APF.L',
 'AV.L',
 'AZN.L',
 'BARC.L',
 'BATS.L',
 'BLT.L',
 'CCL.L',
 'CEY.L',
 'CNA.L',
 'CPG.L',
 'IOG.L',
 'ITV.L',
 'KGF.L',
 'LAND.L',
 'LGEN.L',
 'MAB.L',
 'MKS.L',
 'NG.L',
 'PRU.L',
 'PSON.L',
 'RB.L',
 'RBS.L',
 'RDSa.L',
 'RDSb.L',
 'REL.L',
 'RR.L',
 'RSA.L',
 'RTO.L',
 'SDR.L',
 'SGE.L',
 'SHP.L',
 'SMIN.L',
 'SPT.L',
 'STAN.L',
 'TSCO.L',
 'ULVR.L',
 'UU.L',
 'VOD.L',
 'WPP.L']

In [55]:
# label_symbol_date_path = os.path.join(fileutils.data_path, LabelsAlternateNames[alternate_label_value_idex],
#                                                       ticker, dateIdex)


'/media/ak/My Passport/Data/FinDataReal/'

In [41]:
symbol = symbols[0]
symbol_pickle_file = [f for f in labels_pickle_files if str(symbol) in f][0]

data = pkl.load(open("/".join((dataDrive, symbol_pickle_file)), "rb"),
                                                encoding='latin1')
symbol_pickle_file

'AAL.L_AlternateLabelsLocations.pkl'

In [42]:

MKLExpPath = "/".join((MKLPath, symbol, 'Experiments'))  # symbol experiment path
MKLSymbolPath = "/".join((MKLPath, symbol))  # symbol path
MKLKernPath = "/".join((MKLPath, symbol, 'Kernels'))  # symbol kernel path


## lets extract some dates FOR LABELS

In [43]:
labels_dates_keys = list(data.keys())
labels_dict=[]
for idx,_ in enumerate(labels_dates_keys):
    for labelIdx,_ in enumerate(LabelsAlternateNames):
        labels_dict.append((data[labels_dates_keys[idx]][LabelsAlternateNames[labelIdx]].split("/")[-1]).split(".")[0])

### list of locations for each alternative label

In [44]:
labelsOneAlternateLocations = {labels_dates_keys[s]:data[labels_dates_keys[s]][LabelsAlternateNames[0]] for s in range(0,len(labels_dates_keys))}
labelsTwoAlternateLocations = {labels_dates_keys[s]:data[labels_dates_keys[s]][LabelsAlternateNames[1]] for s in range(0,len(labels_dates_keys))}
labelsThreeAlternateLocations = {labels_dates_keys[s]:data[labels_dates_keys[s]][LabelsAlternateNames[2]] for s in range(0,len(labels_dates_keys))}
labelsFourAlternateLocations = {labels_dates_keys[s]:data[labels_dates_keys[s]][LabelsAlternateNames[3]] for s in range(0,len(labels_dates_keys))}
labelsFourAlternateLocations = {labels_dates_keys[s]:data[labels_dates_keys[s]][LabelsAlternateNames[4]] for s in range(0,len(labels_dates_keys))}


### specific symbol targets locations

In [45]:


symbolFeaturesLocation = "/".join((featuresPath, symbol, 'MODEL_BASED'))
#symbolLabelsLocation = ## need to think how I deal with this one
symbolHMMDatesList = os.listdir(symbolFeaturesLocation)
# list of all the MODEL dates we have generated features files for. each #
# each of these dates in symbolFeaturesDates corresponds to a list of dates
# (symbolHMMDatesList = '20170829', '20170710', '20170801', ... ]
symbolLabelsDates =  list(set(labelsTwoAlternateLocations.keys())) # true dates of data labels --> which correspond for all the dates

In [46]:
set(symbolHMMDatesList)==set(symbolLabelsDates) #dates match

True

## keep track of various structs

In [47]:
# now lets go down into each HMM-model date, and pick all the forward futures (out of sample)
hmmFeatureLocations = {}  # symbol-hmm-model-date index --> this is the indexation in symbolFeaturesDatesList
commonDatesDict = {}  # this is a struct that will contain for each HMM date, the common labels/features- this
# should
# be used for training and testing
createDate = []  # place holder for the hash key of when the features got created
createDateTwo = '20181229'
symbolEachModelFeaturesDates = {}
HMMModelFeaturesLabelsCommon = {}  # location dictionary with 2 keys: HMM Date and Common Date
commonDates = []

LocDictsList = []  # symbol specific

In [14]:
for hmmDateIdx, hmmDate in enumerate(sorted(symbolHMMDatesList)):
    symbolModelFeaturesDate = os.path.join(symbolFeaturesLocation, symbolHMMDatesList[hmmDateIdx])
    if len(os.listdir(symbolModelFeaturesDate)) != 0:
        create_date = os.listdir(symbolModelFeaturesDate)[0].split("_")[-2]
        # output looks like this: /media/ak/DataOnly/FinDataReal/PRU.L/MODEL_BASED/20170710
        # output is a dictionary where the keys are the HMM models dates and the values a list of dates - for
        # each HMM date we have a list of features
      

In [15]:
create_date

'20181227'

In [16]:
symbolEachModelKeys = list(symbolEachModelFeaturesDates.keys())

In [25]:
# symbolEachModelFeaturesDates[symbolEachModelKeys[2]]
symbolEachModelKeys

[]

In [18]:
## get model common dates for each model
os.path.join('/media/ak/DataOnly/FinDataReal/', symbol)

'/media/ak/DataOnly/FinDataReal/AAL.L'

In [22]:
for keyHMMDate in sorted(list(symbolEachModelFeaturesDates.keys())):  # for each of the HMM model dates
    common_dates = list(set(symbolEachModelFeaturesDates[keyHMMDate]).intersection(symbolLabelsDates))
    # take the list of feature dates (conditional on HMM model date) + the list of labels -intersection!

    '''we now produce a dict for each HMM model, where each value is a list of common dates and we are key-ed by
    the HMM Date '''
    commonDatesDict[keyHMMDate] = common_dates
    for commonDate in common_dates:
        print(commonDate)
        labelsCommonFileLoc = labelsOneAlternateLocations[commonDate]
        FeatureFileLoc = os.path.join('/media/ak/DataOnly/FinDataReal/', symbol,'MODEL_BASED',commonDate,symbolEachModelFeaturesDates[keyHMMDate][commonDate])
        conditions = [os.path.isfile(FeatureFileLoc), os.path.isfile(labelsCommonFileLoc)]
        print(conditions)
#

In [24]:
#pd.read_csv(labelsOneAlternateLocations[commonDate])
os.path.isfile(labelsCommonFileLoc)

NameError: name 'labelsCommonFileLoc' is not defined

In [109]:
FeatureFileLoc

'/media/ak/DataOnly/FinDataReal/APF.L/MODEL_BASED/20170131/APF.L_3_states_features_date:_20170131_now:_20190109_.pickle'

In [111]:
os.listdir('/media/ak/DataOnly/FinDataReal/APF.L/MODEL_BASED/20170131/')

['APF.L_3_states_features_date:_20170808_now:_20190109_.pickle',
 'APF.L_3_states_features_date:_20170809_now:_20190109_.pickle',
 'APF.L_3_states_features_date:_20170901_now:_20181229_.pickle',
 'APF.L_3_states_features_date:_20170919_now:_20181229_.pickle',
 'APF.L_3_states_features_date:_20170704_now:_20190109_.pickle',
 'APF.L_3_states_features_date:_20170808_now:_20181229_.pickle',
 'APF.L_3_states_features_date:_20170809_now:_20181229_.pickle',
 'APF.L_3_states_features_date:_20170816_now:_20190109_.pickle',
 'APF.L_3_states_features_date:_20170714_now:_20181229_.pickle',
 'APF.L_3_states_features_date:_20170822_now:_20181229_.pickle',
 'APF.L_3_states_features_date:_20170801_now:_20181229_.pickle',
 'APF.L_3_states_features_date:_20170719_now:_20181229_.pickle',
 'APF.L_3_states_features_date:_20170908_now:_20181229_.pickle',
 'APF.L_3_states_features_date:_20170915_now:_20190109_.pickle',
 'APF.L_3_states_features_date:_20170713_now:_20190109_.pickle',
 'APF.L_3_states_features

In [ ]:
# commonDatesKeys  =  list(commonDatesDict.keys())  # each of these dates correspond to an HMM model
# commonDatesKeys
# for commonDateKey in common:

In [34]:
conditions

[False, False]

need to work out the next set

In [ ]:
for hmmModelIdxDate,_ in enumerate(commonDatesKeys): # lets iterate through all the HMM models and for each one pick the common dates of models and labels
    print('-------------------------------')
    print(hmmDateIdx)
    print('-------------------------------')
    common_dates = commonDatesDict[hmmModelIdxDate] # pick the list of common dates
#     print(common_dates)
# #     for commonDate in common_dates:
# # #         '''iterate through all the common dates and figure out the location of each file for labels and
# # #         features '''
# #         labelsCommonFileLoc = labelsOneAlternateLocations[commonDate]
# #         print(symbolEachModelFeaturesDates[keyHMMDate])
    
# # #         print(labelsCommonFileLoc)


# # # #         conditions = [os.path.exists(FeatureFileLoc), os.path.exists(labelsCommonFileLoc)]


In [ ]:
commonDate
# commonDatesFeatureFile = "".join(
#                             (
#                             symbols[symbol], '_3_states_features_date:_', commonDate, "_now:_", create_date, "_.pickle"))
#     FeatureFileLoc = os.path.join(symbolModelFeaturesDate, commonDatesFeatureFile)

In [ ]:
common_dates